# Demand Forecasting Analysis

## Introduction
**Objective**: Analyse past 2 years of e-commerce operations data, in order to build and optimize a forecasting model for the next 4, 6 and 8 weeks, specific to:
-  Daily Order Volume
- Daily customer care tickets

## Initiation

### Import Libraries

### Import Dataset

### Data Cleaning

## Outlier Detection and Analysis

### Outlier Identification

#### IQR Approach

#### z-score Approach

### Outlier Visualization

### Observation Summary (Handling of Outliers)

## Feature Engineering

### Create Relevant Feature

### Correlation Analysis

## Data Preparation 

### Data Transformation

### Set up Training & Testing Data

## Model Build 

### Model 1

### Model 2

## Model Evaluation